# 広告情報取得

In [ ]:
# 必要なパッケージ
# pip3 install openpyxl xlwt

In [17]:
from config import *
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re
import time
import pandas as pd

In [18]:
def click_start(driver):
    """
    広告を再生するために動画の再生ボタンを押す
    """
    xpath = '//*[@id="movie_player"]/div[8]/button'
    element = driver.find_elements(by=By.XPATH, value=xpath)
    try:
        element[0].click()
        print("click")
    except Exception as e:
        print(e)

def get_video_info(videoID):
    """
    指定したvideoIDの詳細情報を取得
    """

    params = {
        "id": videoID,
        "key": YOUTUBE_API_KEY,
        "part": 'snippet,statistics,status,contentDetails,topicDetails',
        # "fields": "id,snippet,statistics,status",
    }

    r = requests.get(URL_VIDEOS, params=params)
    result_dic = {}
    result_dic["viewCount"] = r.json()["items"][0]["statistics"].get('viewCount',"不明")# 再生回数
    result_dic["commentCount"] = r.json()["items"][0]["statistics"].get('commentCount',"不明")# コメント数
    result_dic["likeCount"] = r.json()["items"][0]["statistics"].get('likeCount',"不明")# いいねの数
    result_dic["tags"] = ",".join(r.json()["items"][0]["snippet"].get("tags","不明"))# タグ
    result_dic["description"] = r.json()["items"][0]["snippet"].get("description","不明")# 動画詳細
    result_dic["title"] = r.json()["items"][0]["snippet"].get("title","不明")#動画タイトル 
    result_dic["channelTitle"] = r.json()["items"][0]["snippet"].get("channelTitle","不明")# チャンネル名
    return result_dic
    

In [19]:
video_search_words = "ポケモン"

cm_num = 5
video_num = 10

params = {
    "key": YOUTUBE_API_KEY,
    "part": 'snippet',
    "q":video_search_words,
    "order":'viewCount',
    "type":'video',
    "maxResults": video_num
    # "fields": "id,snippet,statistics,status",
}
r = requests.get(URL_SEARCH, params=params)
  
# top_bideoID = r.json()["items"][0]["id"]["videoId"]

result_df = pd.DataFrame()
for n,video in enumerate(r.json()["items"][:video_num]):
    
    options = Options()
    # linux上で動かすときは次の2つも実行してください。
    options.add_argument('--headless')
    options.add_argument('--incognito') #シークレットモード

    driver = webdriver.Chrome(options=options)
    videoID = video['id']["videoId"]
    print(n,videoID)

    url = f"https://www.youtube.com/watch?v={videoID}"
    driver.get(url)
    url_list = []
    count_num = 0
    print("開始")
    while True:
        driver.refresh() # ページの更新
        time.sleep(5)
        click_start(driver) #動画再生（広告の割り込み）
        time.sleep(3)

        # 広告URLが格納されているxpath
        xpath = '//*[@id="companion"]/top-banner-image-text-icon-buttoned-layout-view-model/div[2]/ad-button-view-model/a'
        result = driver.find_elements(by=By.XPATH, value=xpath)

        # 取得した要素のHTMLを表示
        for element in result:
            # url_search_list = re.findall('adurl=(.*)', element.get_attribute("href"))
            url_search_list = element.get_attribute("href")
            if len(url_search_list) > 0:
                url_list.append(url_search_list)
                url_num = len(url_list)
                count_num = 0

        if (len(url_list)>5) or (count_num==9):
            break
        count_num += 1

    url_list = list(set(url_list))
    video_info_dic = get_video_info(videoID)
    video_info_dic["koukoku"]= ",".join(url_list)
    a = pd.Series(video_info_dic, name=f"rank{n}")
        
    result_df = pd.concat([result_df, a.to_frame().T])
    # driverを終了する
    driver.quit()

0 oP-CWIgeY9E
開始
click
click
click
click
click
click
click
click
click
click
click
1 zlRDOj-3EpA
開始
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
2 BoZ0Zwab6Oc
開始
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
3 xtkcVp43Df4
開始
click
click
click
click
click
click
click
click
click
click
click
click
click
4 qYllGk_5gl0
開始
click
click
click
click
click
click
click
click
click
click
5 1X9HkJNIRfM
開始
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of rang

In [22]:
result_df.to_excel("result.xlsx")

In [23]:
result_df.shape

(10, 8)